In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
class NameConventionMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        for name in clsdict:
            if name.lower() != name:
                raise TypeError(f'bad attribute name: {name}')

        return super().__new__(cls, clsname, bases, clsdict)


class Root(metaclass=NameConventionMeta):
    pass


class A(Root):
    def foo_bar(self):
        pass

class B(Root):
    def fooBar(self):
        pass

TypeError: bad attribute name: fooBar

In [16]:
import logging
from inspect import signature


class MatchSignatureMeta(type):
    def __init__(self, clsname, bases, clsdict):
        super().__init__(clsname, bases, clsdict)
        sup = super(self, self)
        for name, value in clsdict.items():
            if name.startswith('_') or not callable(value):
                continue

            prev_dfn = getattr(sup, name, None)
            if prev_dfn:
                prev_sig = signature(prev_dfn)
                val_sig = signature(value)
                if prev_sig != val_sig:
                    msg = f'signature mismatach in {value.__qualname__}; ' \
                          f'{prev_sig} != {val_sig}'
                    logging.warning(msg)


class Root(metaclass=MatchSignatureMeta):
    pass


class A(Root):
    def foo(self, x, y):
        pass

    def spam(self, x, *, z):
        pass


class B(A):
    def foo(self, a, b):
        pass

    def spam(self, x, z):
        pass